# 1. Sinusoid Extract

Notebook to extract the sinusoids of horizontal displacement from mean position for a video file consisting of a visualisation displayed on the lower half of a screen and a laser point projected on the top half of the screen. This can take a long time to execute for a large number of high quality videos. The raw videos employed for measuring the motion to visual latency of <em>The Shapeshifter</em> are several hundred GB in size. Therefore these have not been included in the repository, but can be provided upon request. The extracted sinusoids are included.

## 1. Import Libraries

In [ ]:
import numpy as np
import skvideo
import cv2
from scipy.stats import multivariate_normal
import os
from scipy import signal, stats
import matplotlib.pyplot as plt
import pickle

## 2. Define Functions and Classes

In [ ]:
# Function to find the centre point of an area in an image which is comprised of non-zero value
# Should be applied to images which have been thresholded to isolate an area of interest

def find_centre(img_in):
    
    # Find non-zero pixels
    nons = np.nonzero(img_in)
    
    # If has an area of interest
    if np.any(nons):
        
        # Find vertical and horizontal max
        vmin, vmax, hmin, hmax = nons[0].min(), nons[0].max(), nons[1].min(), nons[1].max()
        
        # Find x, y coordinate of area of interest
        y_coord, x_coord = np.floor((vmax-vmin)/2+vmin), np.floor((hmax-hmin)/2+hmin)
        
    else:
        y_coord, x_coord = None, None
    
    return y_coord, x_coord

In [ ]:
# Class for storing two sinusoid arrays
# One for storing laser point in top half
# One for storing visualisation in bottom halfb

class swing_rec:
    
    def __init__(self, name):
        
        self.name = name
        self.top_array = 0
        self.bottom_array = 0

## 3. Extract Sinusoids for 1, 5, and 10 Position Markers

As videos can be recorded on separate days, with different light conditions and camera positions, the first step is to take the first frame of the first video and use this to set cropping and thresholding parameters

### 3.1. RB1

### 3.1.1. Set variables

In [ ]:
# Set filepath to videos
path = ""

# Select test file for setting extraction parameters
testfile = sorted(os.listdir(path))[0]

### 3.1.2. Load first frame of first video

In [ ]:
# Declare list for video frames
cut = []

# Declare frame counter
counter = 0

# Open video reader
cap = cv2.VideoCapture(path + '/' + testfile)

# Read until video is completed
while(cap.isOpened()):
    
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Place into list
    cut.append(frame)
    
    # Increase frame counter
    counter += 1
    
    # Break after two frames
    if counter > 1:
        break

    # Press q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

# To numpy array
cut=np.array(cut)

In [ ]:
# set crop values - [crop left, crop right, crop top, crop bottom] - values are pixel row or columns

thr = [75, 450,130, 1725] # crop for top half - just laser point
bhr = [450,870,130,1725] # crop for bottom half - just visualisation

In [ ]:
# test top half

plt.figure(figsize=(1080/10,1920/10))
plt.imshow(cut[0,thr[0]:thr[1],thr[2]:thr[3],:])

In [ ]:
# test bottom half

plt.figure(figsize=(1080/10,1920/10))
plt.imshow(cut[0,bhr[0]:bhr[1],bhr[2]:bhr[3],:])

### 3.1.3. Test on extracts

Test extraction on one example from all phases from first and last cycles to ensure parameters return satisfactory sinusoids.

In [ ]:
# Set test parameters

# Declare dictionary for swing_rec objects
rb_1_test_dict = {}

# File extention
file_ex = '.mp4'

# Set threshold values. A heuristic is employed in the code based upon extracting a threshold value from the first frame
# This was successful for around 60% of the videos. For the others, the thresholding value had to be set manually

# top half thresholding
top_thresh_val = 200

# bottom half thresholding
bottom_thresh_val = 200

# Outliers are removed assuming a multivariate normal distribution
# Set outlier percentile threhold
cov_thresh = 0.001

In [ ]:
# Declare list for filenames
filelist = []

# Get filenames - assumes naming convention 'position_markers-cycle-phase-sample.extension'
for file in os.listdir(path):
    
    if '-c1-' in file:
        
        if '01' in file:
            
            filelist.append(file)
            
    if '-c7-' in file:
        
        if '01' in file:
            
            filelist.append(file)
            
print(filelist)

In [ ]:
# Main loop for extracting - run on test files

for file in os.listdir(path):
    
    if file_ex in file:
        
        print(file)

        # Declare list for video frames
        cut = []

        # Open video reader
        cap = cv2.VideoCapture(path + '/' + file)

        # Read until video is completed
        while(cap.isOpened()):
            
          # Capture frame-by-frame
          ret, frame = cap.read()
          if ret == True:

            # Place into list
            cut.append(frame)

            # Press q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
              break

          # Break the loop
          else: 
            break
            
        # When everything done, release the video capture object
        cap.release()

        # Closes all the frames
        cv2.destroyAllWindows()

        # To numpy array
        cut=np.array(cut)

        # Declare array for top half in size of crop
        top_thresh = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[1], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,thr[0]:thr[1],thr[2]:thr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #top_thresh_val

        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,thr[0]:thr[1],thr[2]:thr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            top_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_top = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(top_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_top[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_top[0][0] == 0 and coords_top[0][0]:

                coords_top[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_top[i] = coords_top[i-1]

                else:
                    
                    coords_top[i] = x,y

        # Declare array for top half in size of crop
        bottom_thresh = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[1], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #bottom_thresh_val
        
        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bottom_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_bottom = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(bottom_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_bottom[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_bottom[0][0] == 0 and coords_bottom[0][0]:

                coords_bottom[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_bottom[i] = coords_bottom[i-1]

                else:
                    
                    coords_bottom[i] = x,y

        # Remove outliers from top assuming multivariate normal distibution
        
        # Find covariance matrix for top half coordinates
        covariance_matrix_top = np.cov(coords_top[:,1], coords_top[:,0])
        
        # Find mean values for top half coordinates
        mean_values_top = [np.mean(coords_top[:,1]),np.mean(coords_top[:,0])]
        
        # Create a model for the top half coordinates
        model_top = multivariate_normal(cov=covariance_matrix_top,mean=mean_values_top, allow_singular=True)
        
        # Set percentile threshold
        threshold_top = cov_thresh
        
        # Find values within thresholding value
        outlier_top = model_top.pdf(coords_top) < threshold_top
        
        # Find outlier values
        falses_top = np.where(outlier_top == False)[0]

        
        # Find covariance matrix for bottom half coordinates
        covariance_matrix_bottom = np.cov(coords_bottom[:,1], coords_bottom[:,0])
        
        # Find mean values for bottom half coordinates
        mean_values_bottom = [np.mean(coords_bottom[:,1]),np.mean(coords_bottom[:,0])]
        
        # Create a model for the bottom half coordinates
        model_bottom = multivariate_normal(cov=covariance_matrix_bottom,mean=mean_values_bottom, allow_singular=True)
        
        # Set percentile threshold 
        threshold_bottom = cov_thresh
        
        # Find values within thresholding value
        outlier_bottom = model_bottom.pdf(coords_bottom) < threshold_bottom
        
        # Find outlier values
        falses_bottom = np.where(outlier_bottom == False)[0]

        # Replace outliers with previous value
        for i, coord in enumerate(coords_top):
            
            if i in falses_top:
                
                coords_top[i] = coords_top[i-1]

        for i, coord in enumerate(coords_bottom):
            
            if i in falses_bottom:
                
                coords_bottom[i] = coords_bottom[i-1]

        # Find horizontal means
        centroid_top = np.mean(coords_top[:,1])
        centroid_bottom = np.mean(coords_bottom[:,1])

        # Declare arrays for horizontal displacements
        dist_arr_top = np.zeros(coords_top.shape[0])
        dist_arr_bottom = np.zeros(coords_bottom.shape[0])

        # Calculate horizontal displacements from mean position
        for i in range(dist_arr_top.shape[0]):
            
            dist_arr_top[i] = coords_top[:,1][i] - centroid_top

        for i in range(dist_arr_bottom.shape[0]):
            
            dist_arr_bottom[i] = coords_bottom[:,1][i] - centroid_bottom

        # Normalise and zero mean arrays
        dist_arr_top = dist_arr_top/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_top = dist_arr_top - np.mean(dist_arr_top)
        dist_arr_bottom = dist_arr_bottom/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_bottom = dist_arr_bottom - np.mean(dist_arr_bottom)

        # Declare swing_rec object, set array attributes with arrays and place in dict
        rb_1_test_dict[file] = swing_rec(file)
        rb_1_test_dict[file].top_array = dist_arr_top
        rb_1_test_dict[file].bottom_array = dist_arr_bottom

        del cut
        del top_thresh
        del bottom_thresh

In [ ]:
# plot tests and check if satisfactory sinusoids - if not, try other parameters

for key, item in rb_1_test_dict.items():
    
    fig = plt.figure(figsize=(12, 6))
    
    plt.plot(item.bottom_array, label='Visualisation')
    plt.plot(item.top_array, label='Laser Point')
    
    plt.legend()
    
    plt.title(key)
    
    plt.show()

### 3.1.4. Apply to all

Apply to all videos.

In [ ]:
# Declare dictionary for swing_rec objects
RB_1_Dict = {}

# File extention
file_ex = '.mp4'

# Set threshold values. A heuristic is employed in the code based upon extracting a threshold value from the first frame
# This was successful for around 60% of the videos. For the others, the thresholding value had to be set manually

# top half thresholding
top_thresh_val = 200

# bottom half thresholding
bottom_thresh_val = 200

# Outliers are removed assuming a multivariate normal distribution
# Set outlier percentile threhold
cov_thresh = 0.001

In [ ]:
# Main loop for extracting

for file in os.listdir(path):
    
    if file_ex in file:
        
        print(file)

        # Declare list for video frames
        cut = []

        # Open video reader
        cap = cv2.VideoCapture(path + '/' + file)

        # Read until video is completed
        while(cap.isOpened()):
            
          # Capture frame-by-frame
          ret, frame = cap.read()
          if ret == True:

            # Place into list
            cut.append(frame)

            # Press q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
              break

          # Break the loop
          else: 
            break
            
        # When everything done, release the video capture object
        cap.release()

        # Closes all the frames
        cv2.destroyAllWindows()

        # To numpy array
        cut=np.array(cut)

        # Declare array for top half in size of crop
        top_thresh = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[1], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,thr[0]:thr[1],thr[2]:thr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #top_thresh_val

        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,thr[0]:thr[1],thr[2]:thr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            top_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_top = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(top_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_top[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_top[0][0] == 0 and coords_top[0][0]:

                coords_top[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_top[i] = coords_top[i-1]

                else:
                    
                    coords_top[i] = x,y

        # Declare array for top half in size of crop
        bottom_thresh = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[1], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #bottom_thresh_val
        
        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bottom_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_bottom = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(bottom_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_bottom[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_bottom[0][0] == 0 and coords_bottom[0][0]:

                coords_bottom[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_bottom[i] = coords_bottom[i-1]

                else:
                    
                    coords_bottom[i] = x,y

        # Remove outliers from top assuming multivariate normal distibution
        
        # Find covariance matrix for top half coordinates
        covariance_matrix_top = np.cov(coords_top[:,1], coords_top[:,0])
        
        # Find mean values for top half coordinates
        mean_values_top = [np.mean(coords_top[:,1]),np.mean(coords_top[:,0])]
        
        # Create a model for the top half coordinates
        model_top = multivariate_normal(cov=covariance_matrix_top,mean=mean_values_top, allow_singular=True)
        
        # Set percentile threshold
        threshold_top = cov_thresh
        
        # Find values within thresholding value
        outlier_top = model_top.pdf(coords_top) < threshold_top
        
        # Find outlier values
        falses_top = np.where(outlier_top == False)[0]

        
        # Find covariance matrix for bottom half coordinates
        covariance_matrix_bottom = np.cov(coords_bottom[:,1], coords_bottom[:,0])
        
        # Find mean values for bottom half coordinates
        mean_values_bottom = [np.mean(coords_bottom[:,1]),np.mean(coords_bottom[:,0])]
        
        # Create a model for the bottom half coordinates
        model_bottom = multivariate_normal(cov=covariance_matrix_bottom,mean=mean_values_bottom, allow_singular=True)
        
        # Set percentile threshold 
        threshold_bottom = cov_thresh
        
        # Find values within thresholding value
        outlier_bottom = model_bottom.pdf(coords_bottom) < threshold_bottom
        
        # Find outlier values
        falses_bottom = np.where(outlier_bottom == False)[0]

        # Replace outliers with previous value
        for i, coord in enumerate(coords_top):
            
            if i in falses_top:
                
                coords_top[i] = coords_top[i-1]

        for i, coord in enumerate(coords_bottom):
            
            if i in falses_bottom:
                
                coords_bottom[i] = coords_bottom[i-1]

        # Find horizontal means
        centroid_top = np.mean(coords_top[:,1])
        centroid_bottom = np.mean(coords_bottom[:,1])

        # Declare arrays for horizontal displacements
        dist_arr_top = np.zeros(coords_top.shape[0])
        dist_arr_bottom = np.zeros(coords_bottom.shape[0])

        # Calculate horizontal displacements from mean position
        for i in range(dist_arr_top.shape[0]):
            
            dist_arr_top[i] = coords_top[:,1][i] - centroid_top

        for i in range(dist_arr_bottom.shape[0]):
            
            dist_arr_bottom[i] = coords_bottom[:,1][i] - centroid_bottom

        # Normalise and zero mean arrays
        dist_arr_top = dist_arr_top/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_top = dist_arr_top - np.mean(dist_arr_top)
        dist_arr_bottom = dist_arr_bottom/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_bottom = dist_arr_bottom - np.mean(dist_arr_bottom)

        # Declare swing_rec object, set array attributes with arrays and place in dict
        RB_1_Dict[file] = swing_rec(file)
        RB_1_Dict[file].top_array = dist_arr_top
        RB_1_Dict[file].bottom_array = dist_arr_bottom

        del cut
        del top_thresh
        del bottom_thresh

### 3.1.5. Save out to pickle

In [ ]:
with open('./sine_dicts_01.pkl', 'wb') as pickle_file:
    pickle.dump(RB_1_Dict, pickle_file)

### 3.2. RB5

### 3.2.1. Set variables

In [ ]:
# Set filepath to videos
path = ""

# Select test file for setting extraction parameters
testfile = sorted(os.listdir(path))[0]

### 3.2.2. Load first frame of first video

In [ ]:
# Declare list for video frames
cut = []

# Declare frame counter
counter = 0

# Open video reader
cap = cv2.VideoCapture(path + '/' + testfile)

# Read until video is completed
while(cap.isOpened()):
    
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Place into list
    cut.append(frame)
    
    # Increase frame counter
    counter += 1
    
    # Break after two frames
    if counter > 1:
        break

    # Press q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

# To numpy array
cut=np.array(cut)

In [ ]:
# set crop values - [crop left, crop right, crop top, crop bottom] - values are pixel row or columns

thr = [75, 450,130, 1725] # crop for top half - just laser point
bhr = [450,870,130,1725] # crop for bottom half - just visualisation

In [ ]:
# test top half

plt.figure(figsize=(1080/10,1920/10))
plt.imshow(cut[0,thr[0]:thr[1],thr[2]:thr[3],:])

In [ ]:
# test bottom half

plt.figure(figsize=(1080/10,1920/10))
plt.imshow(cut[0,bhr[0]:bhr[1],bhr[2]:bhr[3],:])

### 3.2.3. Test on extracts

Test extraction on one example from all phases from first and last cycles to ensure parameters return satisfactory sinusoids.

In [ ]:
# Set test parameters

# Declare dictionary for swing_rec objects
rb_5_test_dict = {}

# File extention
file_ex = '.mp4'

# Set threshold values. A heuristic is employed in the code based upon extracting a threshold value from the first frame
# This was successful for around 60% of the videos. For the others, the thresholding value had to be set manually

# top half thresholding
top_thresh_val = 200

# bottom half thresholding
bottom_thresh_val = 200

# Outliers are removed assuming a multivariate normal distribution
# Set outlier percentile threhold
cov_thresh = 0.001

In [ ]:
# Declare list for filenames
filelist = []

# Get filenames - assumes naming convention 'position_markers-cycle-phase-sample.extension'
for file in os.listdir(path):
    
    if '-c1-' in file:
        
        if '01' in file:
            
            filelist.append(file)
            
    if '-c7-' in file:
        
        if '01' in file:
            
            filelist.append(file)
            
print(filelist)

In [ ]:
# Main loop for extracting - run on test files

for file in os.listdir(path):
    
    if file_ex in file:
        
        print(file)

        # Declare list for video frames
        cut = []

        # Open video reader
        cap = cv2.VideoCapture(path + '/' + file)

        # Read until video is completed
        while(cap.isOpened()):
            
          # Capture frame-by-frame
          ret, frame = cap.read()
          if ret == True:

            # Place into list
            cut.append(frame)

            # Press q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
              break

          # Break the loop
          else: 
            break
            
        # When everything done, release the video capture object
        cap.release()

        # Closes all the frames
        cv2.destroyAllWindows()

        # To numpy array
        cut=np.array(cut)

        # Declare array for top half in size of crop
        top_thresh = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[1], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,thr[0]:thr[1],thr[2]:thr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #top_thresh_val

        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,thr[0]:thr[1],thr[2]:thr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            top_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_top = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(top_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_top[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_top[0][0] == 0 and coords_top[0][0]:

                coords_top[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_top[i] = coords_top[i-1]

                else:
                    
                    coords_top[i] = x,y

        # Declare array for top half in size of crop
        bottom_thresh = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[1], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #bottom_thresh_val
        
        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bottom_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_bottom = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(bottom_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_bottom[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_bottom[0][0] == 0 and coords_bottom[0][0]:

                coords_bottom[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_bottom[i] = coords_bottom[i-1]

                else:
                    
                    coords_bottom[i] = x,y

        # Remove outliers from top assuming multivariate normal distibution
        
        # Find covariance matrix for top half coordinates
        covariance_matrix_top = np.cov(coords_top[:,1], coords_top[:,0])
        
        # Find mean values for top half coordinates
        mean_values_top = [np.mean(coords_top[:,1]),np.mean(coords_top[:,0])]
        
        # Create a model for the top half coordinates
        model_top = multivariate_normal(cov=covariance_matrix_top,mean=mean_values_top, allow_singular=True)
        
        # Set percentile threshold
        threshold_top = cov_thresh
        
        # Find values within thresholding value
        outlier_top = model_top.pdf(coords_top) < threshold_top
        
        # Find outlier values
        falses_top = np.where(outlier_top == False)[0]

        
        # Find covariance matrix for bottom half coordinates
        covariance_matrix_bottom = np.cov(coords_bottom[:,1], coords_bottom[:,0])
        
        # Find mean values for bottom half coordinates
        mean_values_bottom = [np.mean(coords_bottom[:,1]),np.mean(coords_bottom[:,0])]
        
        # Create a model for the bottom half coordinates
        model_bottom = multivariate_normal(cov=covariance_matrix_bottom,mean=mean_values_bottom, allow_singular=True)
        
        # Set percentile threshold 
        threshold_bottom = cov_thresh
        
        # Find values within thresholding value
        outlier_bottom = model_bottom.pdf(coords_bottom) < threshold_bottom
        
        # Find outlier values
        falses_bottom = np.where(outlier_bottom == False)[0]

        # Replace outliers with previous value
        for i, coord in enumerate(coords_top):
            
            if i in falses_top:
                
                coords_top[i] = coords_top[i-1]

        for i, coord in enumerate(coords_bottom):
            
            if i in falses_bottom:
                
                coords_bottom[i] = coords_bottom[i-1]

        # Find horizontal means
        centroid_top = np.mean(coords_top[:,1])
        centroid_bottom = np.mean(coords_bottom[:,1])

        # Declare arrays for horizontal displacements
        dist_arr_top = np.zeros(coords_top.shape[0])
        dist_arr_bottom = np.zeros(coords_bottom.shape[0])

        # Calculate horizontal displacements from mean position
        for i in range(dist_arr_top.shape[0]):
            
            dist_arr_top[i] = coords_top[:,1][i] - centroid_top

        for i in range(dist_arr_bottom.shape[0]):
            
            dist_arr_bottom[i] = coords_bottom[:,1][i] - centroid_bottom

        # Normalise and zero mean arrays
        dist_arr_top = dist_arr_top/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_top = dist_arr_top - np.mean(dist_arr_top)
        dist_arr_bottom = dist_arr_bottom/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_bottom = dist_arr_bottom - np.mean(dist_arr_bottom)

        # Declare swing_rec object, set array attributes with arrays and place in dict
        rb_5_test_dict[file] = swing_rec(file)
        rb_5_test_dict[file].top_array = dist_arr_top
        rb_5_test_dict[file].bottom_array = dist_arr_bottom

        del cut
        del top_thresh
        del bottom_thresh

In [ ]:
# plot tests and check if satisfactory sinusoids - if not, try other parameters

for key, item in rb_5_test_dict.items():
    
    fig = plt.figure(figsize=(12, 6))
    
    plt.plot(item.bottom_array, label='Visualisation')
    plt.plot(item.top_array, label='Laser Point')
    
    plt.legend()
    
    plt.title(key)
    
    plt.show()

### 3.2.4. Apply to all

Apply to all videos.

In [ ]:
# Declare dictionary for swing_rec objects
RB_5_Dict = {}

# File extention
file_ex = '.mp4'

# Set threshold values. A heuristic is employed in the code based upon extracting a threshold value from the first frame
# This was successful for around 60% of the videos. For the others, the thresholding value had to be set manually

# top half thresholding
top_thresh_val = 200

# bottom half thresholding
bottom_thresh_val = 200

# Outliers are removed assuming a multivariate normal distribution
# Set outlier percentile threhold
cov_thresh = 0.001

In [ ]:
# Main loop for extracting

for file in os.listdir(path):
    
    if file_ex in file:
        
        print(file)

        # Declare list for video frames
        cut = []

        # Open video reader
        cap = cv2.VideoCapture(path + '/' + file)

        # Read until video is completed
        while(cap.isOpened()):
            
          # Capture frame-by-frame
          ret, frame = cap.read()
          if ret == True:

            # Place into list
            cut.append(frame)

            # Press q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
              break

          # Break the loop
          else: 
            break
            
        # When everything done, release the video capture object
        cap.release()

        # Closes all the frames
        cv2.destroyAllWindows()

        # To numpy array
        cut=np.array(cut)

        # Declare array for top half in size of crop
        top_thresh = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[1], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,thr[0]:thr[1],thr[2]:thr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #top_thresh_val

        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,thr[0]:thr[1],thr[2]:thr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            top_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_top = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(top_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_top[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_top[0][0] == 0 and coords_top[0][0]:

                coords_top[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_top[i] = coords_top[i-1]

                else:
                    
                    coords_top[i] = x,y

        # Declare array for top half in size of crop
        bottom_thresh = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[1], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #bottom_thresh_val
        
        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bottom_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_bottom = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(bottom_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_bottom[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_bottom[0][0] == 0 and coords_bottom[0][0]:

                coords_bottom[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_bottom[i] = coords_bottom[i-1]

                else:
                    
                    coords_bottom[i] = x,y

        # Remove outliers from top assuming multivariate normal distibution
        
        # Find covariance matrix for top half coordinates
        covariance_matrix_top = np.cov(coords_top[:,1], coords_top[:,0])
        
        # Find mean values for top half coordinates
        mean_values_top = [np.mean(coords_top[:,1]),np.mean(coords_top[:,0])]
        
        # Create a model for the top half coordinates
        model_top = multivariate_normal(cov=covariance_matrix_top,mean=mean_values_top, allow_singular=True)
        
        # Set percentile threshold
        threshold_top = cov_thresh
        
        # Find values within thresholding value
        outlier_top = model_top.pdf(coords_top) < threshold_top
        
        # Find outlier values
        falses_top = np.where(outlier_top == False)[0]

        
        # Find covariance matrix for bottom half coordinates
        covariance_matrix_bottom = np.cov(coords_bottom[:,1], coords_bottom[:,0])
        
        # Find mean values for bottom half coordinates
        mean_values_bottom = [np.mean(coords_bottom[:,1]),np.mean(coords_bottom[:,0])]
        
        # Create a model for the bottom half coordinates
        model_bottom = multivariate_normal(cov=covariance_matrix_bottom,mean=mean_values_bottom, allow_singular=True)
        
        # Set percentile threshold 
        threshold_bottom = cov_thresh
        
        # Find values within thresholding value
        outlier_bottom = model_bottom.pdf(coords_bottom) < threshold_bottom
        
        # Find outlier values
        falses_bottom = np.where(outlier_bottom == False)[0]

        # Replace outliers with previous value
        for i, coord in enumerate(coords_top):
            
            if i in falses_top:
                
                coords_top[i] = coords_top[i-1]

        for i, coord in enumerate(coords_bottom):
            
            if i in falses_bottom:
                
                coords_bottom[i] = coords_bottom[i-1]

        # Find horizontal means
        centroid_top = np.mean(coords_top[:,1])
        centroid_bottom = np.mean(coords_bottom[:,1])

        # Declare arrays for horizontal displacements
        dist_arr_top = np.zeros(coords_top.shape[0])
        dist_arr_bottom = np.zeros(coords_bottom.shape[0])

        # Calculate horizontal displacements from mean position
        for i in range(dist_arr_top.shape[0]):
            
            dist_arr_top[i] = coords_top[:,1][i] - centroid_top

        for i in range(dist_arr_bottom.shape[0]):
            
            dist_arr_bottom[i] = coords_bottom[:,1][i] - centroid_bottom

        # Normalise and zero mean arrays
        dist_arr_top = dist_arr_top/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_top = dist_arr_top - np.mean(dist_arr_top)
        dist_arr_bottom = dist_arr_bottom/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_bottom = dist_arr_bottom - np.mean(dist_arr_bottom)

        # Declare swing_rec object, set array attributes with arrays and place in dict
        RB_5_Dict[file] = swing_rec(file)
        RB_5_Dict[file].top_array = dist_arr_top
        RB_5_Dict[file].bottom_array = dist_arr_bottom

        del cut
        del top_thresh
        del bottom_thresh

### 3.2.5. Save out to pickle

In [ ]:
with open('./sine_dicts_05.pkl', 'wb') as pickle_file:
    pickle.dump(RB_5_Dict, pickle_file)

### 3.3. RB10

### 3.3.1. Set variables

In [ ]:
# Set filepath to videos
path = ""

# Select test file for setting extraction parameters
testfile = sorted(os.listdir(path))[0]

### 3.3.2. Load first frame of first video

In [ ]:
# Declare list for video frames
cut = []

# Declare frame counter
counter = 0

# Open video reader
cap = cv2.VideoCapture(path + '/' + testfile)

# Read until video is completed
while(cap.isOpened()):
    
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Place into list
    cut.append(frame)
    
    # Increase frame counter
    counter += 1
    
    # Break after two frames
    if counter > 1:
        break

    # Press q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

# To numpy array
cut=np.array(cut)

In [ ]:
# set crop values - [crop left, crop right, crop top, crop bottom] - values are pixel row or columns

thr = [75, 450,130, 1725] # crop for top half - just laser point
bhr = [450,870,130,1725] # crop for bottom half - just visualisation

In [ ]:
# test top half

plt.figure(figsize=(1080/10,1920/10))
plt.imshow(cut[0,thr[0]:thr[1],thr[2]:thr[3],:])

In [ ]:
# test bottom half

plt.figure(figsize=(1080/10,1920/10))
plt.imshow(cut[0,bhr[0]:bhr[1],bhr[2]:bhr[3],:])

### 3.3.3. Test on extracts

Test extraction on one example from all phases from first and last cycles to ensure parameters return satisfactory sinusoids.

In [ ]:
# Set test parameters

# Declare dictionary for swing_rec objects
rb_10_test_dict = {}

# File extention
file_ex = '.mp4'

# Set threshold values. A heuristic is employed in the code based upon extracting a threshold value from the first frame
# This was successful for around 60% of the videos. For the others, the thresholding value had to be set manually

# top half thresholding
top_thresh_val = 200

# bottom half thresholding
bottom_thresh_val = 200

# Outliers are removed assuming a multivariate normal distribution
# Set outlier percentile threhold
cov_thresh = 0.001

In [ ]:
# Declare list for filenames
filelist = []

# Get filenames - assumes naming convention 'position_markers-cycle-phase-sample.extension'
for file in os.listdir(path):
    
    if '-c1-' in file:
        
        if '01' in file:
            
            filelist.append(file)
            
    if '-c7-' in file:
        
        if '01' in file:
            
            filelist.append(file)
            
print(filelist)

In [ ]:
# Main loop for extracting - run on test files

for file in os.listdir(path):
    
    if file_ex in file:
        
        print(file)

        # Declare list for video frames
        cut = []

        # Open video reader
        cap = cv2.VideoCapture(path + '/' + file)

        # Read until video is completed
        while(cap.isOpened()):
            
          # Capture frame-by-frame
          ret, frame = cap.read()
          if ret == True:

            # Place into list
            cut.append(frame)

            # Press q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
              break

          # Break the loop
          else: 
            break
            
        # When everything done, release the video capture object
        cap.release()

        # Closes all the frames
        cv2.destroyAllWindows()

        # To numpy array
        cut=np.array(cut)

        # Declare array for top half in size of crop
        top_thresh = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[1], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,thr[0]:thr[1],thr[2]:thr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #top_thresh_val

        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,thr[0]:thr[1],thr[2]:thr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            top_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_top = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(top_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_top[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_top[0][0] == 0 and coords_top[0][0]:

                coords_top[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_top[i] = coords_top[i-1]

                else:
                    
                    coords_top[i] = x,y

        # Declare array for top half in size of crop
        bottom_thresh = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[1], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #bottom_thresh_val
        
        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bottom_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_bottom = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(bottom_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_bottom[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_bottom[0][0] == 0 and coords_bottom[0][0]:

                coords_bottom[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_bottom[i] = coords_bottom[i-1]

                else:
                    
                    coords_bottom[i] = x,y

        # Remove outliers from top assuming multivariate normal distibution
        
        # Find covariance matrix for top half coordinates
        covariance_matrix_top = np.cov(coords_top[:,1], coords_top[:,0])
        
        # Find mean values for top half coordinates
        mean_values_top = [np.mean(coords_top[:,1]),np.mean(coords_top[:,0])]
        
        # Create a model for the top half coordinates
        model_top = multivariate_normal(cov=covariance_matrix_top,mean=mean_values_top, allow_singular=True)
        
        # Set percentile threshold
        threshold_top = cov_thresh
        
        # Find values within thresholding value
        outlier_top = model_top.pdf(coords_top) < threshold_top
        
        # Find outlier values
        falses_top = np.where(outlier_top == False)[0]

        
        # Find covariance matrix for bottom half coordinates
        covariance_matrix_bottom = np.cov(coords_bottom[:,1], coords_bottom[:,0])
        
        # Find mean values for bottom half coordinates
        mean_values_bottom = [np.mean(coords_bottom[:,1]),np.mean(coords_bottom[:,0])]
        
        # Create a model for the bottom half coordinates
        model_bottom = multivariate_normal(cov=covariance_matrix_bottom,mean=mean_values_bottom, allow_singular=True)
        
        # Set percentile threshold 
        threshold_bottom = cov_thresh
        
        # Find values within thresholding value
        outlier_bottom = model_bottom.pdf(coords_bottom) < threshold_bottom
        
        # Find outlier values
        falses_bottom = np.where(outlier_bottom == False)[0]

        # Replace outliers with previous value
        for i, coord in enumerate(coords_top):
            
            if i in falses_top:
                
                coords_top[i] = coords_top[i-1]

        for i, coord in enumerate(coords_bottom):
            
            if i in falses_bottom:
                
                coords_bottom[i] = coords_bottom[i-1]

        # Find horizontal means
        centroid_top = np.mean(coords_top[:,1])
        centroid_bottom = np.mean(coords_bottom[:,1])

        # Declare arrays for horizontal displacements
        dist_arr_top = np.zeros(coords_top.shape[0])
        dist_arr_bottom = np.zeros(coords_bottom.shape[0])

        # Calculate horizontal displacements from mean position
        for i in range(dist_arr_top.shape[0]):
            
            dist_arr_top[i] = coords_top[:,1][i] - centroid_top

        for i in range(dist_arr_bottom.shape[0]):
            
            dist_arr_bottom[i] = coords_bottom[:,1][i] - centroid_bottom

        # Normalise and zero mean arrays
        dist_arr_top = dist_arr_top/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_top = dist_arr_top - np.mean(dist_arr_top)
        dist_arr_bottom = dist_arr_bottom/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_bottom = dist_arr_bottom - np.mean(dist_arr_bottom)

        # Declare swing_rec object, set array attributes with arrays and place in dict
        rb_10_test_dict[file] = swing_rec(file)
        rb_10_test_dict[file].top_array = dist_arr_top
        rb_10_test_dict[file].bottom_array = dist_arr_bottom

        del cut
        del top_thresh
        del bottom_thresh

In [ ]:
# plot tests and check if satisfactory sinusoids - if not, try other parameters

for key, item in rb_10_test_dict.items():
    
    fig = plt.figure(figsize=(12, 6))
    
    plt.plot(item.bottom_array, label='Visualisation')
    plt.plot(item.top_array, label='Laser Point')
    
    plt.legend()
    
    plt.title(key)
    
    plt.show()

### 3.1.4. Apply to all

Apply to all videos.

In [ ]:
# Declare dictionary for swing_rec objects
RB_10_Dict = {}

# File extention
file_ex = '.mp4'

# Set threshold values. A heuristic is employed in the code based upon extracting a threshold value from the first frame
# This was successful for around 60% of the videos. For the others, the thresholding value had to be set manually

# top half thresholding
top_thresh_val = 200

# bottom half thresholding
bottom_thresh_val = 200

# Outliers are removed assuming a multivariate normal distribution
# Set outlier percentile threhold
cov_thresh = 0.001

In [ ]:
# Main loop for extracting

for file in os.listdir(path):
    
    if file_ex in file:
        
        print(file)

        # Declare list for video frames
        cut = []

        # Open video reader
        cap = cv2.VideoCapture(path + '/' + file)

        # Read until video is completed
        while(cap.isOpened()):
            
          # Capture frame-by-frame
          ret, frame = cap.read()
          if ret == True:

            # Place into list
            cut.append(frame)

            # Press q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
              break

          # Break the loop
          else: 
            break
            
        # When everything done, release the video capture object
        cap.release()

        # Closes all the frames
        cv2.destroyAllWindows()

        # To numpy array
        cut=np.array(cut)

        # Declare array for top half in size of crop
        top_thresh = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[1], cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,thr[0]:thr[1],thr[2]:thr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #top_thresh_val

        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,thr[0]:thr[1],thr[2]:thr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            top_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_top = np.zeros((cut[:,thr[0]:thr[1],thr[2]:thr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(top_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_top[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_top[0][0] == 0 and coords_top[0][0]:

                coords_top[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_top[i] = coords_top[i-1]

                else:
                    
                    coords_top[i] = x,y

        # Declare array for top half in size of crop
        bottom_thresh = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[1], cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[2]), dtype=np.uint8)
        
        # Convert first frame to greyscale
        thresh_first = cv2.cvtColor(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:][0], cv2.COLOR_BGR2GRAY)
        
        # Heuristic of setting threshold to half the max value. If doesn't work, has to be set manually.
        thresh_val = thresh_first.max()/2 #bottom_thresh_val
        
        # Convert entire video to greyscale and threshold
        for i, frame in enumerate(cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:]):
            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bottom_thresh[i] = cv2.threshold(thresh, thresh_val, 255, cv2.THRESH_BINARY)[1]

        # Declare array for centre coordinates 
        coords_bottom = np.zeros((cut[:,bhr[0]:bhr[1],bhr[2]:bhr[3],:].shape[0], 2))

        # Extract centre coordinates of area of interest
        for i, frame in enumerate(bottom_thresh):
            
            # Find centre of area of interest
            x, y = find_centre(frame)
            
            # For first frame
            if i == 0:

                # Leave zero if no centre found
                if x == None or y == None:

                    pass

                else:

                    coords_bottom[i] = x,y

            # At finding of first non zero, set previous to this value
            if i > 0 and coords_bottom[0][0] == 0 and coords_bottom[0][0]:

                coords_bottom[0] = x, y

            if i != 0:
                
                # If current frame centre not found, set previous value
                if x == None or y == None:
                    
                    coords_bottom[i] = coords_bottom[i-1]

                else:
                    
                    coords_bottom[i] = x,y

        # Remove outliers from top assuming multivariate normal distibution
        
        # Find covariance matrix for top half coordinates
        covariance_matrix_top = np.cov(coords_top[:,1], coords_top[:,0])
        
        # Find mean values for top half coordinates
        mean_values_top = [np.mean(coords_top[:,1]),np.mean(coords_top[:,0])]
        
        # Create a model for the top half coordinates
        model_top = multivariate_normal(cov=covariance_matrix_top,mean=mean_values_top, allow_singular=True)
        
        # Set percentile threshold
        threshold_top = cov_thresh
        
        # Find values within thresholding value
        outlier_top = model_top.pdf(coords_top) < threshold_top
        
        # Find outlier values
        falses_top = np.where(outlier_top == False)[0]

        
        # Find covariance matrix for bottom half coordinates
        covariance_matrix_bottom = np.cov(coords_bottom[:,1], coords_bottom[:,0])
        
        # Find mean values for bottom half coordinates
        mean_values_bottom = [np.mean(coords_bottom[:,1]),np.mean(coords_bottom[:,0])]
        
        # Create a model for the bottom half coordinates
        model_bottom = multivariate_normal(cov=covariance_matrix_bottom,mean=mean_values_bottom, allow_singular=True)
        
        # Set percentile threshold 
        threshold_bottom = cov_thresh
        
        # Find values within thresholding value
        outlier_bottom = model_bottom.pdf(coords_bottom) < threshold_bottom
        
        # Find outlier values
        falses_bottom = np.where(outlier_bottom == False)[0]

        # Replace outliers with previous value
        for i, coord in enumerate(coords_top):
            
            if i in falses_top:
                
                coords_top[i] = coords_top[i-1]

        for i, coord in enumerate(coords_bottom):
            
            if i in falses_bottom:
                
                coords_bottom[i] = coords_bottom[i-1]

        # Find horizontal means
        centroid_top = np.mean(coords_top[:,1])
        centroid_bottom = np.mean(coords_bottom[:,1])

        # Declare arrays for horizontal displacements
        dist_arr_top = np.zeros(coords_top.shape[0])
        dist_arr_bottom = np.zeros(coords_bottom.shape[0])

        # Calculate horizontal displacements from mean position
        for i in range(dist_arr_top.shape[0]):
            
            dist_arr_top[i] = coords_top[:,1][i] - centroid_top

        for i in range(dist_arr_bottom.shape[0]):
            
            dist_arr_bottom[i] = coords_bottom[:,1][i] - centroid_bottom

        # Normalise and zero mean arrays
        dist_arr_top = dist_arr_top/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_top = dist_arr_top - np.mean(dist_arr_top)
        dist_arr_bottom = dist_arr_bottom/max(np.abs(dist_arr_top.max()),np.abs(dist_arr_top.min()),np.abs(dist_arr_bottom.max()),np.abs(dist_arr_bottom.min()))
        dist_arr_bottom = dist_arr_bottom - np.mean(dist_arr_bottom)

        # Declare swing_rec object, set array attributes with arrays and place in dict
        RB_10_Dict[file] = swing_rec(file)
        RB_10_Dict[file].top_array = dist_arr_top
        RB_10_Dict[file].bottom_array = dist_arr_bottom

        del cut
        del top_thresh
        del bottom_thresh

### 3.1.5. Save out to pickle

In [ ]:
with open('./sine_dicts_10.pkl', 'wb') as pickle_file:
    pickle.dump(RB_10_Dict, pickle_file)